In [1]:
# Generative Adversarial Network using TensorFlow
# TODO: save checkpoints

#import o tensorflow
import tensorflow as tf

In [2]:
import numpy as np #import o numpy

In [3]:
import matplotlib.pyplot as plt # importa o matplotlib para fazer gráficos
import matplotlib.gridspec as gridspec

In [4]:
import os # importa pacotes do sistema operacional

In [5]:
from tensorflow.examples.tutorials.mnist import input_data # importa o dataset mnist

In [6]:
# inicializa os pesos

In [7]:
# Xavier init is well explained here in Andy's blog:
# http://andyljones.tumblr.com/post/110998971763/an-explanation-of-xavier-initialization
#
# Another good explanation can be found in Prateek Joshi's blog:
# https://prateekvjoshi.com/2016/03/29/understanding-xavier-initialization-in-deep-neural-networks/
def xavier_init(size):
    input_dim = size[0]
    xavier_variance = 1. / tf.sqrt(input_dim/2.)
    return tf.random_normal(shape=size, stddev=xavier_variance)

In [9]:
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig

In [10]:
# Generator Net
Z = tf.placeholder(tf.float32, shape=[None, 100], name='Z') # o ruído da entrada do gerador

In [11]:
G_W1 = tf.Variable(xavier_init([100, 128]), name='G_W1') # peso da primeira camada
G_b1 = tf.Variable(tf.zeros(shape=[128]), name='G_b1')   # bias da primeira camada

In [12]:
G_W2 = tf.Variable(xavier_init([128, 784]), name='G_W2') #peso da segunda camada
G_b2 = tf.Variable(tf.zeros(shape=[784]), name='G_b2')   # bias da segunda camada

In [13]:
theta_G = [G_W1, G_W2, G_b1, G_b2] #parametros do gerador. Uma lista dos pesos e bias acima

In [14]:
# Discriminator Net
X = tf.placeholder(tf.float32, shape=[None, 784], name='X') # a entrada real do discriminador

In [15]:
D_W1 = tf.Variable(xavier_init([784, 128]), name='D_W1') #peso da primeira camada do discriminador
D_b1 = tf.Variable(tf.zeros(shape=[128]), name='D_b1') # bias da primeira camada do discriminador

In [16]:
D_W2 = tf.Variable(xavier_init([128, 1]), name='D_W2') #pesoa da segunda camada do discriminador
D_b2 = tf.Variable(tf.zeros(shape=[1]), name='D_b2') #bias da segunda camada do discriminador

In [17]:
theta_D = [D_W1, D_W2, D_b1, D_b2] #parametros do discriminador : lsita dos pesos e bias

In [18]:
def generator(z):  # entra com o ruído z
    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1) #relu sobre o (z * G_W1 + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2    # G_h1 * G_W2 + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob) #sigmoid sobre a camada anterior

    return G_prob

In [19]:
def discriminator(x): # entra com o dado real
    D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1) # da mesma forma que o gerador X * D_W1 + D_b1. Só muda a ent X
    D_logit = tf.matmul(D_h1, D_W2) + D_b2       # da mesma forma que o gerador D_logit = D_h1 * D_W2 + D_b2 
    D_prob = tf.nn.sigmoid(D_logit)              # sigmoid

    return D_prob, D_logit  #retorna tbm o logit antes de passar pela camada de ativação

In [20]:
G_sample = generator(Z) # chama o gerador

In [21]:
D_real, D_logit_real = discriminator(X) # retorna o dado real e logit real

In [22]:
D_fake, D_logit_fake = discriminator(G_sample) #passa o dado fake para o discriminador

In [23]:
# Loss functions from the paper
# D_loss = -tf.reduce_mean(tf.log(D_real) + tf.log(1. - D_fake))
# G_loss = -tf.reduce_mean(tf.log(D_fake))

In [24]:
# Alternative loss functions
D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_real, labels=tf.ones_like(D_logit_real)))

In [25]:
D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.zeros_like(D_logit_fake)))

In [26]:
D_loss = D_loss_real + D_loss_fake #soma as duas loss real e fake

In [27]:
G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.ones_like(D_logit_fake)))

In [28]:
#G_loss = D_loss_fake

In [29]:
# Update D(X)'s parameters
D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D) # atualiza os parametros da lista. Pesos e Bias

In [30]:
# Update G(Z)'s parameters
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)

In [31]:
def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])

In [32]:
batch_size = 128
Z_dim = 100

In [33]:

sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [34]:
mnist = input_data.read_data_sets('MNIST/', one_hot=True)


Extracting MNIST/train-images-idx3-ubyte.gz
Extracting MNIST/train-labels-idx1-ubyte.gz
Extracting MNIST/t10k-images-idx3-ubyte.gz
Extracting MNIST/t10k-labels-idx1-ubyte.gz


In [35]:
if not os.path.exists('output/'):
    os.makedirs('output/')

i = 0

In [36]:
for itr in range(1000000):
    if itr % 1000 == 0:
        samples = sess.run(G_sample, feed_dict={Z: sample_Z(16, Z_dim)})

        fig = plot(samples)
        plt.savefig('output/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
        i += 1
        plt.close(fig)

    X_mb, _ = mnist.train.next_batch(batch_size)

    _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict={X: X_mb, Z: sample_Z(batch_size, Z_dim)})
    _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={Z: sample_Z(batch_size, Z_dim)})

    if itr % 1000 == 0:
        print('Iter: {}'.format(itr))
        print('D loss: {:.4}'. format(D_loss_curr))
        print('G_loss: {:.4}'.format(G_loss_curr))
        print()

Iter: 0
D loss: 1.269
G_loss: 2.636

Iter: 1000
D loss: 0.005969
G_loss: 6.681

Iter: 2000
D loss: 0.02154
G_loss: 5.56

Iter: 3000
D loss: 0.05964
G_loss: 6.317

Iter: 4000
D loss: 0.2623
G_loss: 6.478

Iter: 5000
D loss: 0.4929
G_loss: 4.528

Iter: 6000
D loss: 0.4321
G_loss: 3.958

Iter: 7000
D loss: 0.5772
G_loss: 4.421

Iter: 8000
D loss: 0.5597
G_loss: 3.808

Iter: 9000
D loss: 0.5188
G_loss: 3.926

Iter: 10000
D loss: 0.8118
G_loss: 3.243

Iter: 11000
D loss: 0.9817
G_loss: 2.677

Iter: 12000
D loss: 0.6872
G_loss: 2.778

Iter: 13000
D loss: 0.6767
G_loss: 2.456

Iter: 14000
D loss: 0.8266
G_loss: 2.223

Iter: 15000
D loss: 0.9554
G_loss: 2.674

Iter: 16000
D loss: 0.6446
G_loss: 2.316

Iter: 17000
D loss: 0.5969
G_loss: 3.072

Iter: 18000
D loss: 0.731
G_loss: 2.469

Iter: 19000
D loss: 0.7655
G_loss: 2.128

Iter: 20000
D loss: 0.7863
G_loss: 2.849

Iter: 21000
D loss: 0.6826
G_loss: 2.098

Iter: 22000
D loss: 0.99
G_loss: 1.744

Iter: 23000
D loss: 0.7495
G_loss: 2.011

Iter: 

KeyboardInterrupt: 